This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')
print(API_key)

NameError: name 'API_key' is not defined

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [ ]:
# First, import the relevant modules
import requests
import json
import statistics

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [ ]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = "https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X.json?start_date=2014-01-01&end_date=2014-1-02&api_key=NASDAQ_API_KEY=4-PJ17R4Jacy6ReY2GEX"
r = requests.get(url)
jsonfile = r.json()

In [ ]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(jsonfile)

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [ ]:
#Task 1 and 2: Collecting 2017 data for AFX_X and converting it into a Python Dictionary
url = "https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31&api_key=NASDAQ_API_KEY=4-PJ17R4Jacy6ReY2GEX"
r = requests.get(url)
json2017 = r.json()
print(type(json2017))

In [ ]:
#Task 3: Calculate the highest and lowest opening prices for the stock in this period
list = json2017["dataset"]['data'] #initial dataset that we are working with json2017["dataset"]['data']
list2 = [i[0:2] for i in list] #only keep 'Date' and 'Open' columns
OpenPrices = [i[1] for i in list2] #only keep 'Open' column
for i, v in enumerate(OpenPrices): #Removing Nones by replacing them with the previous value(which is the next day's opening price) ideally you would replace the missing opening price with the previous day's closing price
    if v is None:
        OpenPrices[i] = OpenPrices[i-1]
Date = [i[0] for i in list2] #only keep 'Date' column
maxindex = OpenPrices.index(max(OpenPrices)) #find where the maximum value is located
minindex = OpenPrices.index(min(OpenPrices)) #find where the minimum value is located
print(f"The day with the highest opening price was {Date[maxindex]}. The opening price on this date was {OpenPrices[maxindex]}.")
print(f"The day with the lowest opening price was {Date[minindex]}. The opening price on this date was {OpenPrices[minindex]}.")

In [ ]:
#Task 4: Calculate the largest change in any one day (Based on High and Low Price)
ChangeInitialList = [i[0:4] for i in list] #Filtering out first 4 columns
Dates = [i[0] for i in ChangeInitialList] #Filtering out date
Change = [i[2]-i[3] for i in ChangeInitialList] #Finding difference between highest price and lowest price per day
maxChangeindex = Change.index(max(Change)) #find where the maximum value is located
print(f"The day with the largest change in price was {Dates[maxChangeindex]}. The change in price on this date was {Change[maxChangeindex]}.")

In [ ]:
#Task 5: Calculate the largest change between any two days (based on Closing Price)
Close = [i[4] for i in list] #Filtering out the 'Close' column
DailyChange = [] 
for i in range(len(Close)-1): #creating a new list called DailyChange which has the values of today's close minus yesterday's close
    TodaysChange = Close[i] - Close[i+1]
    DailyChange.append(TodaysChange)
print(f"The largest change between any two days was {max(DailyChange)}.")
print(f"The largest change between any two days was {min(DailyChange)}.")

In [ ]:
#Task 6: Calculate the Average Daily Trading Volume for this year
DailyTraded = [i[6] for i in list] #Filtering out the 'Traded Volume' column
print(f"The Average Daily Trading Volume for this year was {statistics.mean(DailyTraded)}.")

In [ ]:
#Task 7: Calculate the Median Daily Trading Volume for this year
print(f"The Median Daily Traded Volume for this year was {statistics.median(DailyTraded)}.")